In [1]:
class A:
    def method(self):
        print('method of A')
        
class B:
    def method(self):
        print('method of B')

In [2]:
class C(A, B):
    pass

In [3]:
c = C()

In [4]:
c.method()

method of A


In [5]:
class D(B, A):
    pass

In [6]:
d = D()

In [7]:
d.method()

method of B


**多个类继承的时候，谁在前继承谁**

In [8]:
class E(A):
    def method(self):
        print('method of E')

In [9]:
class F(A, E):
    pass

TypeError: Cannot create a consistent method resolution
order (MRO) for bases A, E

In [10]:
class G(E, A):
    pass

In [11]:
g = G()

In [12]:
g.method()

method of E


MRO 特性:
- 本地优先：自己定义或 重写的方法 优先，按照继承列表，从左到右查找
- 单调性：所有子类，也要满足查找顺序。 G 继承了 E 和 A， 那么G 既要满足 E 的MRO， 又要满足 A 的MRO

In [13]:
A.__mro__

(__main__.A, object)

In [14]:
E.__mro__

(__main__.E, __main__.A, object)

In [15]:
G.__mro__

(__main__.G, __main__.E, __main__.A, object)

(F, A, E, object)

F, A, E , A

python 通过C3 算法 来完成 MRO 顺序查找

```
class B(O) -> [B, O]

class B(A1, A2, A3, ..., An) -> [B] + merge(mro(A1), mro(A2), mro(A3), ..., mro(An), [A1, A2, A3, ..., An, O])
```

- 遍历列表
- 看第一个列表的首元素
    - 在其他列表中 也是首元素
    - 在其他列表中 不存在
- 如果满足以上2种情况，移出，并合并到MRO
- 如果不满足，抛出异常

```
mro(G) -> [G] + merge(mro(E), mro(A), [E, A, O])

     -> [G] + merge([E, A, O], [A, O], [E, A, O])
     
     -> [G, E] + merge([A, O], [A, O], [A, O])
     
     -> [G, E, A] + merge([O], [O], [O])
     
     -> [G, E, A, O]
```

```
mro(F) -> [F] + merge(mro(A), mro(E), [A, E, O])

    -> [F] + merge([A, O], [E, A, O], [A, E, O])
    
    -> [F] + merge([A])
```

In [16]:
class A:
    pass

class  B:
    pass

class C(A):
    pass

class D(B, A):
    pass

class E(C, A):
    pass

class F(E):
    pass

class G(D, F):
    pass

类装饰器可以干什么？
- 设置一些类变量，但是仅仅是设置类变量 是比较少的
- 可以给类增加一些方法，但是呢，用类装饰器 给类增加方法不是特别好

真正给类 添加一类方法的时候，我们会使用Mixin

## Mixin

```
        Document
        
    Word        Excel
    
PrintWord            PrintExcel

PrintPaperWord PrintPrinterWord

```

In [18]:
def print_to_printer(cls):
    def print(self):
        pass
    
    cls.print_to_printer = print # 动态的给类 添加 print 方法
    return cls

但是类装饰器 存在如下缺陷：

如果一个类 不能动态的 添加成员的时候

确实存在这么一种方法，可以规避 不能动态添加成员

In [20]:
class Printable:
    def print(self):
        pass

In [21]:
class Document:
    def __init__(self, content):
        self.content = content
        
class Word(Document):
    def __init__(self, content):
        new_content = 'word: {}'.format(content)
        super().__init__(new_content)
        
class Excel(Document):
    def __init__(self, content):
        super().__init__('excel: {}'.format(content))

In [22]:
def printable(cls):
    def _print(self):
        print(self)
        print(self.content)
        
    cls.print = _print
    return cls

In [23]:
@printable
class PrintableWord(Word):
    def __init__(self, content):
        super().__init__(content)

In [24]:
pw = PrintableWord('abc')

In [25]:
pw.print()

word: abc


In [26]:
def printable(cls):
    def _print(self):
        print('P: {}'.format(self.content))
        
    cls.print = _print
    return cls

In [27]:
@printable
class PrintableWord(Word):
    def __init__(self, content):
        super().__init__(content)

In [28]:
pw = PrintableWord('abc')

In [29]:
pw.print()

P: word: abc


In [30]:
def print_to_monitor(cls):
    def _print(self):
        print('Monitor : P: {}'.format(self.content))
    
    cls.print = _print
    
    return cls

In [32]:
@print_to_monitor
class PrintMonitorWord(Word):
    def __init__(self, content):
        super().__init__(content)

In [33]:
pmw = PrintMonitorWord('abc')

In [34]:
pmw.print()

Monitor : P: word: abc


Mixin

In [45]:
class PrintableMixin:
    def print(self):
        result = 'P: {}'.format(self.content)
        print(result)
        return result

In [46]:
class PrintableWord(PrintableMixin, Word): # 
    def __init__(self, content):
        super().__init__(content) 
        # 这里的content 是一个  传参行为，并不是 继承行为
        # PrintableWord 中的 self.content 是  PrintableWord 的 ， 
                                  # PrintableMixin 里面的 self.content 是 PrintableWord 的
            # PrintableWord 的 content 是从 Word 那里 得到的参数， 并不是继承过来的

In [37]:
pw = PrintableWord('abc')

In [38]:
pw.print()

P: word: abc


In [47]:
class PrintToMonitorMixin(PrintableMixin):
    def print(self):
        print('Monitor: {}'.format(super().print()))
        
class PrintToMonitorWord(PrintToMonitorMixin, Word):
    def __init__(self, content):
        super().__init__(content)

In [48]:
pmw = PrintToMonitorWord('abc')

In [49]:
pmw.print()

P: word: abc
Monitor: P: word: abc


Mixin 的限制：
- Mixin 类不应该有 初始化方法 `__init__`
- Mixin 类 通常不能独立工作
- Mixin 类 的祖先也应该是Mixin 类

通常来说，Mixin 类 需要放在继承列表的 第一位

- 如果Mixin 有明确的继承关系，我们可以用
- 如果不能有明确的继承关系，我们就用装饰器来做

## 面相对象2 高级

魔术方法 / 专有方法

In [50]:
class A:
    pass

In [51]:
dir(A)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']

In [52]:
a = A()

In [53]:
dir(a)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']

In [55]:
A.__name__

'A'

In [56]:
A.__module__

'__main__'

In [57]:
A.__doc__

In [58]:
A.__class__

type

In [59]:
a.__name__

AttributeError: 'A' object has no attribute '__name__'

In [60]:
a.__doc__

In [61]:
a.__module__

'__main__'

In [62]:
a.__class__

__main__.A

In [63]:
a.__class__.__name__

'A'

In [64]:
a.__dict__ # 实例的所有属性 都保存在 __dict__ 里面

{}

In [65]:
a.xxx = 3

In [66]:
a.__dict__

{'xxx': 3}

In [67]:
a.__dict__['xxx'] = 3

In [68]:
a.__dict__

{'xxx': 3}

In [69]:
a.__dict__['xxx'] = 5 # 实例的属性 为什么可以动态添加的原因

In [70]:
a.__dict__

{'xxx': 5}

In [72]:
hash(set([1, 2, 3]))

TypeError: unhashable type: 'set'

In [73]:
a.__dict__[set([1, 2, 3])] = 5

TypeError: unhashable type: 'set'

In [74]:
a.__dict__['x x x'] = 5

In [75]:
a.x x x

SyntaxError: invalid syntax (<ipython-input-75-7b9f7a82be6c>, line 1)

In [76]:
a.__dict__['x x x']

5

In [77]:
a.__dict__

{'x x x': 5, 'xxx': 5}

In [78]:
dir(a)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'x x x',
 'xxx']

In [79]:
a.__dict__

{'x x x': 5, 'xxx': 5}

In [84]:
sorted(a.__dir__())

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'x x x',
 'xxx']

dir() 方法是  `__dir__()` 排序后的结果

反射

分类：

- 创建/ 销毁 （init 函数）
- 运算符重载
- hash
- bool
- 可视化
- 反射
- 上下文管理
- 大小
- 描述器
- 杂项

## 运算重载符

In [85]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [86]:
a = Point(1, 2)
b = Point(3, 4)

In [87]:
a + b # (4, 6)

TypeError: unsupported operand type(s) for +: 'Point' and 'Point'

In [88]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def add(self, other):
        return Point(self.x + other.x, self.y + other.y)

In [96]:
a = Point(1, 2)
b = Point(3, 4)

In [91]:
c = a.add(b)

In [93]:
c.x

4

In [94]:
c.y

6

In [95]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __add__(self, other):
        return Point(self.x + other.x, self.y + other.y)

In [97]:
c = a + b

In [98]:
c

In [99]:
c.x

4

In [100]:
c.y

6

In [101]:
help(int)

Help on class int in module builtins:

class int(object)
 |  int(x=0) -> integer
 |  int(x, base=10) -> integer
 |  
 |  Convert a number or string to an integer, or return 0 if no arguments
 |  are given.  If x is a number, return x.__int__().  For floating point
 |  numbers, this truncates towards zero.
 |  
 |  If x is not a number or if base is given, then x must be a string,
 |  bytes, or bytearray instance representing an integer literal in the
 |  given base.  The literal can be preceded by '+' or '-' and be surrounded
 |  by whitespace.  The base defaults to 10.  Valid bases are 0 and 2-36.
 |  Base 0 means to interpret the base from the string as an integer literal.
 |  >>> int('0b100', base=0)
 |  4
 |  
 |  Methods defined here:
 |  
 |  __abs__(self, /)
 |      abs(self)
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __and__(self, value, /)
 |      Return self&value.
 |  
 |  __bool__(self, /)
 |      self != 0
 |  
 |  __ceil__(...)
 |      Ceiling of

In [102]:
t = 1

In [103]:
t +=1

In [104]:
t

2

In [105]:
c += Point(10, 10)

In [106]:
c.x

14

In [107]:
c.y

16

### 算术运算

In [112]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __add__(self, other):
        return Point(self.x + other.x, self.y + other.y)
    
    def __sub__(self, other):
        return Point(self.x - other.x, self.y - other.y)
    
    def __mul__(self, other):
        return Point(self.x * other.x, self.y * other.y)

In [113]:
p = Point(3, 5) * Point(2, 4)

In [114]:
p.x

6

In [115]:
p.y

20

### 位运算符

### 成员运算符

In [116]:
1 in [1,2, 3]

True

In [117]:
help(list)

Help on class list in module builtins:

class list(object)
 |  list() -> new empty list
 |  list(iterable) -> new list initialized from iterable's items
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __delitem__(self, key, /)
 |      Delete self[key].
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(...)
 |      x.__getitem__(y) <==> x[y]
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __iadd__(self, value, /)
 |      Implement self+=value.
 |  
 |  __imul__(self, value, /)
 |      Implement self*=value.
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __l

In [118]:
i = 3

In [119]:
i + 4

7

In [120]:
i.__add__(4)

7

In [121]:
help(set)

Help on class set in module builtins:

class set(object)
 |  set() -> new empty set object
 |  set(iterable) -> new set object
 |  
 |  Build an unordered collection of unique elements.
 |  
 |  Methods defined here:
 |  
 |  __and__(self, value, /)
 |      Return self&value.
 |  
 |  __contains__(...)
 |      x.__contains__(y) <==> y in x.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __iand__(self, value, /)
 |      Return self&=value.
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __ior__(self, value, /)
 |      Return self|=value.
 |  
 |  __isub__(self, value, /)
 |      Return self-=value.
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __ixor__(self, value, /)
 |      Re

## 第八周课后练习

1. 实现LinkedList
2. 实现优先队列

（使用面向对象的 思想 去解决）